In [40]:
import numpy as np
import pandas as pd
import re
import string

from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [41]:
train_df = pd.read_csv(r"D:\Personal\Kaggle\Tweet_disaster\train.csv")
test_df = pd.read_csv(r"D:\Personal\Kaggle\Tweet_disaster\test.csv")
sample_submission = pd.read_csv(r"D:\Personal\Kaggle\Tweet_disaster\sample_submission.csv")

In [42]:
print(train_df.head(1))

   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   

   target  
0       1  


In [43]:
print(test_df.head(1))

   id keyword location                                text
0   0     NaN      NaN  Just happened a terrible car crash


In [45]:
count_vectorizer = feature_extraction.text.CountVectorizer()
## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [46]:
## we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space)
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


In [17]:
#train_vectors = count_vectorizer.fit_transform(train_df["text"])
## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
#test_vectors = count_vectorizer.transform(test_df["text"])

In [47]:
#TF-IDF VECTORIZER
tfidf_vectorizer = feature_extraction.text.TfidfVectorizer()
train_vectors = tfidf_vectorizer.fit_transform(train_df["text"])
test_vectors = tfidf_vectorizer.transform(test_df["text"])

In [48]:
## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
clf = linear_model.RidgeClassifier()

In [49]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.63441451, 0.6122449 , 0.68407835])

In [50]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [51]:
sample_submission["target"] = clf.predict(test_vectors)

In [52]:
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [53]:
sample_submission.to_csv("submission.csv", index=False)